In [1]:
from dotenv import load_dotenv
import json
import os
import tweepy
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk.tokenize import word_tokenize
from gensim.parsing.preprocessing import STOPWORDS
import gensim
import string
import nltk
import sparknlp
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /common/home/dw659/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
load_dotenv()

True

In [3]:
bearer_token = os.environ.get('bearer_token')
consumer_key = os.environ.get('consumer_key')
consumer_secret = os.environ.get('consumer_secret')
access_token = os.environ.get('access_token')
access_token_secret = os.environ.get('access_token_secret')

In [4]:
client = tweepy.Client(
    bearer_token=bearer_token,
    consumer_key=consumer_key,
    consumer_secret=consumer_secret,
    access_token=access_token,
    access_token_secret=access_token_secret,
)

In [5]:
# Get Twitter ID from handle from https://tweeterid.com/
accounts = [
    {
        'handle': 'cnnbrk',
        'id': 428333
    },
    {
        'handle': 'CNN',
        'id': 759251
    },
    {
        'handle': 'nytimes',
        'id': 807095
    },
    {
        'handle': 'BBCBreaking',
        'id': 5402612
    },
    {
        'handle': 'BBCWorld',
        'id': 742143
    },
    {
        'handle': 'TheEconomist',
        'id': 5988062
    },
    {
        'handle': 'WSJ',
        'id': 3108351
    },
    {
        'handle': 'washingtonpost',
        'id': 2467791
    },
    {
        'handle': 'TIME',
        'id': 14293310
    },
    {
        'handle': 'ABC',
        'id': 28785486
    },
    {
        'handle': 'ndtv',
        'id': 37034483
    },
    {
        'handle': 'AP',
        'id': 51241574
    },
    {
        'handle': 'XHNews',
        'id': 487118986
    },
    {
        'handle': 'HuffPost',
        'id': 14511951
    },
    {
        'handle': 'guardian',
        'id': 87818409
    },
    {
        'handle': 'BreakingNews',
        'id': 6017542
    },
    {
        'handle': 'SkyNews',
        'id': 7587032
    },
    {
        'handle': 'AJEnglish',
        'id': 4970411
    },
    {
        'handle': 'FT',
        'id': 18949452
    },
    {
        'handle': 'SkyNewsBreak',
        'id': 87416722
    },
    {
        'handle': 'politico',
        'id': 9300262
    },
    {
        'handle': 'CNBC',
        'id': 20402945
    },
    {
        'handle': 'FRANCE24',
        'id': 1994321
    },
    {
        'handle': 'guardiannews',
        'id': 788524
    },
    {
        'handle': 'Independent',
        'id': 16973333
    },
    {
        'handle': 'BBCAfrica',
        'id': 36670025
    },
    {
        'handle': 'Newsweek',
        'id': 2884771
    },
    {
        'handle': 'Telegraph',
        'id': 16343974
    },
    {
        'handle': 'RT_com',
        'id': 64643056
    },
    {
        'handle': 'CBCNews',
        'id': 6433472
    },
    {
        'handle': 'FinancialTimes',
        'id': 4898091
    },
    {
        'handle': 'Reuters',
        'id': 1652541
    },
    {
        'handle': 'SportsCenter',
        'id': 26257166
    },
    {
        'handle': 'espn',
        'id': 2557521
    }
]

In [6]:
def get_tweets(accounts, n):
    tweets = []
    for account in accounts:
        response = client.get_users_tweets(account['id'], max_results=n)
        for tweet in response.data:
            tweets.append(tweet.text)
    return tweets

In [ ]:
tweets = get_tweets(accounts, 10)

In [ ]:
tokenized_words = []
def lemmatize_stemming(text):
    return WordNetLemmatizer().lemmatize(text, pos='v')

for tweet in tweets:
    new_tweet = tweet[:tweet.index('http')] if 'http' in tweet else tweet
    new_tweet = re.sub(r'[^a-zA-Z ]+', '', new_tweet)
    tokens = word_tokenize(new_tweet)
    remove_stopwords = [lemmatize_stemming(t.lower()) for t in tokens if not t in STOPWORDS and not t in string.punctuation and len(t) > 2]
    tokenized_words.append(",".join(remove_stopwords))

In [ ]:
tokenized_words[:10